<h1> GENRE: RAP </h1>

<h3>(1) Access the data from the dataset </h3>


In [1]:
%%capture
%pip install datasets

from datasets import load_dataset

ds = load_dataset("sebastiandizon/genius-song-lyrics")

In [9]:
# Assuming the dataset has a 'category' column
unique_categories = ds['train'].unique('tag')  # Get all unique categories

# Group into subdatasets
subdatasets = {category: ds['train'].filter(lambda x: x['tag'] == category and x['language'] == 'en') for category in unique_categories}

type_of_music = 'rap'
# Group into music decades
subdataset_70 = ds['train'].filter(lambda song: 1960 <= song["year"] <= 1970 and song['tag'] == type_of_music)
subdataset_80 = ds['train'].filter(lambda song: 1970 <= song["year"] <= 1980 and song['tag'] == type_of_music)
subdataset_90 = ds['train'].filter(lambda song: 1980 <= song["year"] <= 1990 and song['tag'] == type_of_music)
subdataset_00 = ds['train'].filter(lambda song: 1990 <= song["year"] <= 2000 and song['tag'] == type_of_music)
subdataset_10 = ds['train'].filter(lambda song: 2000 <= song["year"] <= 2010 and song['tag'] == type_of_music)
subdataset_20 = ds['train'].filter(lambda song: 2010 <= song["year"] <= 2020 and song['tag'] == type_of_music)
subdataset_30 = ds['train'].filter(lambda song: 2020 <= song["year"] <= 2030 and song['tag'] == type_of_music)

# Create subdataset
#eng_rap = subdatasets['rap']

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134856 [00:00<?, ? examples/s]

<h2> LDA </h2>

<h2> Remove the bad words</h2>

In [12]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tushifire/ldnoobw")
#file = kagglehub.load_dataset("tushifire/ldnoobw",path)

print("Path to dataset files:", path)

swear_words_file = os.path.join(path, 'en')  # Replace 'en.txt' with the correct filename

with open(swear_words_file, 'r') as f:
    swear_words = set(f.read().splitlines())

print(f"!!!!Loaded {len(swear_words)} swear words.")

Path to dataset files: /home/maurice/.cache/kagglehub/datasets/tushifire/ldnoobw/versions/2
!!!!Loaded 403 swear words.


In [47]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get"])

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
    #doc = remove_stopwords(doc.lower())
    doc = remove_repeated_lines(doc)
    
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    
    return tokens

def remove_repeated_lines(doc):
    lines = doc.split("\n")
    return "\n".join(sorted(set(lines), key=lines.index))

def sort_out_bad_words(doc,word_list, replacement="swear"):
    """
    Ersetzt Wörter in einem Text, wenn sie in einer Liste vorkommen.
    
    :param text: Der Eingabetext (String).
    :param word_list: Liste der Wörter, die ersetzt werden sollen.
    :param replacement: Der Text, durch den die Wörter ersetzt werden sollen.
    :return: Der bearbeitete Text.
    """
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)
 

def process_subdatasets(subdataset):
    # Get rid of the bad words and replace it with "swear word"
    without_swear = [sort_out_bad_words(doc, swear_words ) for doc in subdataset]

    # Preprocessing für alle Dokumente anwenden
    processed_docs = [preprocess(doc) for doc in without_swear]

    # Erstellen eines Wörterbuchs
    dictionary = Dictionary(processed_docs)

    # Erstellen eines Bag-of-Words-Korpus
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    # LDA-Modell trainieren
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=5,  # Anzahl der Themen
        random_state=42,
        passes=40,  # Anzahl der Durchgänge
        alpha='symmetric',
        eta='auto'
        #alpha='auto'  # Dirichlet-Hyperparameter Entscheidung zwischen Auto und symetric
    )
    
    id2word = dictionary
    # Themen anzeigen
    print("Themen und ihre Schlüsselwörter:")
    for idx, topic in lda_model.print_topics(num_words=5):
        print(f"Topic {idx}: {topic}")


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
rap_70_lyrics = process_subdatasets(subdataset_70['lyrics'][:10000])
rap_80_lyrics = process_subdatasets(subdataset_80['lyrics'][:10000])
rap_90_lyrics = process_subdatasets(subdataset_90['lyrics'][:10000])
rap_00_lyrics = process_subdatasets(subdataset_00['lyrics'][:10000])
rap_10_lyrics = process_subdatasets(subdataset_10['lyrics'][:10000])
rap_20_lyrics = process_subdatasets(subdataset_20['lyrics'][:10000])
rap_30_lyrics = process_subdatasets(subdataset_30['lyrics'][:10000])

import csv

# Daten für die CSV
data = [
    {"Method": "LDA", "Genre": "Rap", "Decade": "1970s","Topic": rap_70_lyrics},
    {"Method": "LDA", "Genre": "Rap", "Decade": "1980s","Topic": rap_80_lyrics},
    {"Method": "LDA", "Genre": "Rap", "Decade": "1990s","Topic": rap_90_lyrics},
    {"Method": "LDA", "Genre": "Rap", "Decade": "2000s","Topic": rap_00_lyrics},
    {"Method": "LDA", "Genre": "Rap", "Decade": "2010s","Topic": rap_10_lyrics},
    {"Method": "LDA", "Genre": "Rap", "Decade": "2020s","Topic": rap_20_lyrics},
    {"Method": "LDA", "Genre": "Rap", "Decade": "2030s","Topic": rap_30_lyrics},
]

# Datei schreiben
output_file = "topics_rap_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  # Schreibe die Kopfzeile
    writer.writerows(data)  # Schreibe die Daten

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


Themen und ihre Schlüsselwörter:
Topic 0: 0.040*"yuh" + 0.038*"pib" + 0.038*"flosisisnsn" + 0.038*"bsew" + 0.014*"got"
Topic 1: 0.019*"hey" + 0.014*"got" + 0.011*"yeah" + 0.009*"fortnite" + 0.008*"real"
Topic 2: 0.027*"niggers" + 0.014*"swear" + 0.012*"revolution" + 0.008*"comes" + 0.008*"change"
Topic 3: 0.054*"feat" + 0.012*"swear" + 0.011*"lil" + 0.005*"denzel" + 0.005*"curry"
Topic 4: 0.021*"swear" + 0.015*"kevin" + 0.008*"running" + 0.008*"bitch" + 0.007*"time"
Themen und ihre Schlüsselwörter:
Topic 0: 0.009*"hey" + 0.008*"yeah" + 0.008*"know" + 0.008*"verse" + 0.007*"scott"
Topic 1: 0.027*"jam" + 0.012*"j" + 0.009*"jibbit" + 0.008*"le" + 0.007*"que"
Topic 2: 0.037*"mi" + 0.024*"say" + 0.023*"di" + 0.012*"fi" + 0.011*"cah"
Topic 3: 0.027*"rock" + 0.014*"got" + 0.012*"beat" + 0.011*"said" + 0.011*"say"
Topic 4: 0.020*"swear" + 0.009*"time" + 0.008*"love" + 0.008*"black" + 0.008*"man"
Themen und ihre Schlüsselwörter:
Topic 0: 0.013*"got" + 0.012*"swear" + 0.011*"know" + 0.011*"love"

# tipps von ChaGPT
https://chatgpt.com/c/676f1ffc-b134-8012-9f03-6bd466bda148

- Herausfinden, wieviele Topics am geeingesten sind
- warum sind weiterhin so viele Stop words enthalten
- wieviele Lieder nehm ich überhaupt

In [35]:
print(len(subdataset_80))

440


In [36]:
print(len(subdataset_90))

4505
